In [1]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv


load_dotenv()
github_key = os.getenv("GITHUB_KEY")

In [ ]:
#Get starred 
owner = "rust-lang"
repo = "rust"

headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {github_key}"
    }

base_url = "https://api.github.com"
git_starring = f"/repos/{owner}/{repo}/stargazers"
url = base_url + git_starring

all_results = []
num_pages_requested = 0

while url and num_pages_requested < 400:
        num_pages_requested += 1
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            print(response.headers.get("Link", ""))
            data = response.json()
            
            all_results.extend(data)

            link_header = response.headers.get("Link", "")
            next_url = None
            for link in link_header.split(","):
                if 'rel="next"' in link:
                    next_url = link[link.find("<")+1:link.find(">")]
                    break

            url = next_url
            
        elif response.status_code == 202:
            print("Compiling data, try again shortly")
            break
        else:
            print(f"Error: {response.status_code}")
            break

    # parsing
df = pd.json_normalize(
        all_results, 
        record_path=None, 
        meta=None, 
        errors='ignore'
    )
    # getting what I need
df.head()
df.to_csv("rust_stars.csv", index=False)